# Itertools Walkthrough
----
This notebook is a walkthrough of the project based tutorial on the itertools Python module.  The tutorial is found [here](https://realpython.com/python-itertools/).

In [31]:
from math import sqrt
import itertools as it
import operator
import timeit
import pprint
PP = pprint.PrettyPrinter(indent=4)

## Groupers are tasty fish
----

In [3]:

# Example of low level iterator generators/operators
print(list(zip([1, 2, 3],['a', 'b', 'c'])))
# Zip iterates through each iterator and returns combined tuples of
# ith element in each iteratable object

print(list(map(len, ['abcd', 'ef', 'ghit'])))
# The map function calls the iter() function on an iterable and
# applies the function passed to the value returned by the
# next() function
print(list(map(lambda x: x**2, [2,4,6])))

# Since iterators are iterable, you can combine these functions
print(list(map(sum, zip([1,2,3], [5,6,7]))))

def hypotenuese(args):
    return sqrt(args[0]**2 + args[1]**2)
# Testing out map with multiple inputs on a custom function
# -- TURNS OUT YOU CAN ONLY PASS A SINGLE ARGUMENT, MAKE IT COUNT
print(list(map(hypotenuese, zip([1,2,3],[4,5,6]))))

[(1, 'a'), (2, 'b'), (3, 'c')]
[4, 2, 4]
[4, 16, 36]
[6, 8, 10]
[4.123105625617661, 5.385164807134504, 6.708203932499369]


In [4]:
# Iterators use lazy execution which can drastically improve 
# memory usage and processing speed

# Example - bad option first
def naive_grouper(inputs, n):
    n_groups = len(inputs) // n
    return [tuple(inputs[i*n:(i+1)*n] for i in range(n_groups))]

# This works fine for small lists (to return tuples) but if you 
# pass a list of 100 million numbers it will consume roughly
# 4.5GB of ram and take 11 seconds.

# Let's implement a better solution  - This executes 3-5x as fast and, 
# due to the use of iter(), consumes ~ 630x less RAM
def better_grouper(inputs, n):
    iters = [iter(inputs)]*n
    return zip(*iters)

In [5]:
# Timing the execution of this code.  This is going to take a while.
# Since timeit will execute this code 1M times, we'll keep this short
TEST_CODE = """
for _ in naive_grouper(range(1000), 10):
  pass
"""
print(timeit.timeit(stmt=TEST_CODE, globals=globals()))

44.160402647998126


In [6]:
# Timing the execution of our improved method
TEST_CODE = """
for _ in better_grouper(range(1000), 10):
    pass
"""
print(timeit.timeit(stmt=TEST_CODE, globals=globals()))

19.108165507001104


In [7]:
print(naive_grouper([1,2,3,4,5,6, 7,8,9,10],2))
print(list(better_grouper([1,2,3,4,5,6, 7, 8, 9, 10], 2)))

[([1, 2], [3, 4], [5, 6], [7, 8], [9, 10])]
[(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)]


In [8]:
# better_grouper could still be ...well, better.
# It doesn't had cases where the number of elements is not
# a multiple of the grouping number, e.g.
print("Better Grouper Output, still lacking:\n",
      list(better_grouper([1,2,3,4,5,6, 7, 8, 9, 10], 3)))

# let's see if we can improve on that. We can use the itertools function
# zip_longest().  This will take any number of iterables PLUS a fill
# value that defaults to None
def betterest_grouper(inputs, n, fillvalue=None):
    iters = [iter(inputs)]*n
    return it.zip_longest(*iters, fillvalue=fillvalue)

# Now let's try that again
print("Betterest Grouper can handle this:\n",
      list(betterest_grouper([1,2,3,4,5,6, 7, 8, 9, 10], 3)))

Better Grouper Output, still lacking:
 [(1, 2, 3), (4, 5, 6), (7, 8, 9)]
Betterest Grouper can handle this:
 [(1, 2, 3), (4, 5, 6), (7, 8, 9), (10, None, None)]


### Brute Force seems rude
---

In [11]:
# Interview question:  You have three $20 bills, five $10 bills,
# two $5 bills, and five $1 bills. How many ways can you make change
# for a $100 bill?

# Basic brute force approach.  Iterate through all the possible
# combinations of bills in your wallet and check if they add up
# to $100
bills = [20, 20, 20, 10, 10, 10, 10, 10, 5, 5, 1, 1, 1, 1, 1]
# A choice of k things from a set of n things is called a combination
# itertools has a function for that: itertools.combinations(set, n)

# Let's see all the different combinations of 3 bills we could make
list(it.combinations(bills, 3))

# Nifty but not really helpful, yet. We can add an incrementor for
# bumping up the number of bills in our combination and a check to
# see if it makes 100 to validate our combinations against the 
# original constraint
makes_100=[]
# remember Python starts at 0 but we know 0 bills won't solve the problem
for n in range(1,len(bills)+1):
    for combo in it.combinations(bills, n):
        if sum(combo) == 100:
            makes_100.append(combo)
            
# We use set() to make sure our combos are unique
PP.pprint(set(makes_100))  # And we get 5 unique combinations of bills

# Okay now let's flip this script just a bit.  Try the following question:
# "How many ways are there to make change for a $100 bill using any number
#  of $50, $20, $10, $5, and $1 bills?"

# We don't have a pre-set collection of bills this time so we need to
# generate all possible combinations with any number of the specified bills.
# Thankfully, itertools has combinations_with_replacement(). We just need
# to use an iterator that has all possible bill denominations and then 
# loop over all potential numbers of bills we might use.
bills = [50, 20, 10, 5, 1]  # Generating a list of all denominations we can use.
makes_100 = []
for n in range(1, 101):
    for combo in it.combinations_with_replacement(bills, n):
        if sum(combo) == 100:
            makes_100.append(combo)
            
# PP.pprint(set(makes_100))
# Brute force algorithms can be pretty .... brutal.  The number of outputs
# increases exponentially (factorially actually) with increase in inputs.
# But sometimes you need to consider every possible combination and in
# those cases, itertools has some useful functions.

# Making all possible combinations:
print(it.permutations(['a', 'b', 'c', 'd']))

{   (20, 20, 10, 10, 10, 10, 10, 5, 1, 1, 1, 1, 1),
    (20, 20, 10, 10, 10, 10, 10, 5, 5),
    (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
    (20, 20, 20, 10, 10, 10, 5, 5),
    (20, 20, 20, 10, 10, 10, 10)}


In [22]:
# Numerical Sequences - Some nifty tricks

# First example - Evens and Odds, how to do it without arithmetic.
# We start by looking at arithmetic solutions

def evens():
    n = 0
    while True:
        yield n
        n += 2

def odds():
    n = 1
    while True:
        yield n
        n += 2

evens = evens()
odds = odds()

print(list(next(odds) for _ in range(10)))
print(list(next(evens) for _ in range (10)))

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [29]:
# This works but itertools can make your code more simple and compact.
# We will start with itertools.count()
counter = it.count()
print(list(next(counter) for _ in range(10)))

evens = it.count(step=2)
odds = it.count(start=1, step=2)
print(list(next(odds) for _ in range(10)))
print(list(next(evens) for _ in range (10)))

# While this may seem like more code for the same basic functionality as a
# range() command, the isntantiated counter retains it's position (e.g.)
print(next(odds))
# AND it is an infinite iterator, meaning you do not need to know how many 
# times you need to iterate through it beforehand.
# Example:
print(list(zip(it.count(), ['a', 'b', 'c', 'd'])))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
21
[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd')]


In [34]:
# RECURRENCE RELATIONS - These decribe a sequence of numbers with a recursive
#                        formula (e.g. Fibonacci sequence)
# Fn = Fn-1 + Fn-2; F0 =0, F1=1
# These types of relations are often built with generators (e.g.)
def fibs():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b
# This is a "second order" sequence because you need to look back 2 steps to
# get the next number
# Our previous evens and odds are "first order" recursive sequences because we
# only need to look back 1 step and include a constant.

# More first order examples
count_by_three = it.count(step=3)
count_by_four = it.count(step=4)
# Let's take a look
print(list(next(count_by_three) for _ in range (5)))
print(list(next(count_by_four) for _ in range(5)))

# There are many ways to build these out, tweaking steps, stopping points
# and alternating values.  But let's use the accumulate() function to
# allow us to build any sort of first order sequence

# ACCUMULATE REQUIRES THE FUNCTION PASSED IN IS BINARY, ACCEPTS 2 ARGS.

# Example of how accumulate works
list(it.accumulate([1,2,3,4,5], operator.add))
# You can pass more complex functions to accumulate using lambdas
list(it.accumulate([1,2,3,4,5,6], lambda x,y: sqrt((y+x)**y)))

[0, 3, 6, 9, 12]
[0, 4, 8, 12, 16]


[1,
 3.0,
 14.696938456699069,
 349.5755076535926,
 2367401.0363475108,
 1.3268407533751489e+19]